In [1]:
from sqlalchemy import create_engine, text
import geopandas as gpd

In [2]:
USER='postgres'
PASSWORD='A123456*'
HOST='localhost'
PORT='5433'
DATABASE='utea'

In [3]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
    )

In [4]:
def actualizar_num_liberaciones():
    sentencia_sql = '''
    UPDATE drones_control_bio.numero_liberaciones p
    SET 
    	num_liberaciones = sub.cantidad,
    	num_distintos = sub.distintos,
		area = sub.total_area
    FROM (
        SELECT p.id, COUNT(pol.id) AS cantidad, COUNT(DISTINCT(pol.unidad_05)) AS distintos, sum(pol.area) as total_area
        FROM drones_control_bio.numero_liberaciones p
        JOIN drones_control_bio.parte_diario_ctrl_bio pol
          ON ST_Intersects(p.geom, pol.geom)
    	WHERE pol.estado='EJECUTADO'
        GROUP BY p.id
    ) sub
    WHERE p.id = sub.id
    '''
    with obtener_engine().connect() as conn:
        conn.execute(
            text(sentencia_sql)
        )
        conn.commit()
    print('Se actualizo el numero de apliacion de control biologio.!')

def actualizar_datos():
    sentencia_sql = '''
    UPDATE drones_control_bio.numero_liberaciones p
    SET 
		unidad_01 = (SELECT unidad_01 FROM drones_control_bio.parte_diario_ctrl_bio pol WHERE ST_Intersects(p.geom, pol.geom) LIMIT 1),
		unidad_03 = (SELECT unidad_03 FROM drones_control_bio.parte_diario_ctrl_bio pol WHERE ST_Intersects(p.geom, pol.geom) LIMIT 1),
		unidad_05 = (SELECT unidad_05 FROM drones_control_bio.parte_diario_ctrl_bio pol WHERE ST_Intersects(p.geom, pol.geom) LIMIT 1)
    '''
    with obtener_engine().connect() as conn:
        conn.execute(
            text(sentencia_sql)
        )
        conn.commit()
    print('Se actualizo los datos de numero de liberaciones.!')

In [5]:
actualizar_num_liberaciones()

Se actualizo el numero de apliacion de control biologio.!


In [6]:
actualizar_datos()

Se actualizo los datos de numero de liberaciones.!
